In [2]:
# UI of CP TO CP
from cmath import exp
from operator import le
from tkinter import *
from tkinter import filedialog
import tkinter
from tkinter.ttk import *
from turtle import left, right
from PIL import Image, ImageTk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Paint wafer color in dict hash.
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt



windows = Tk()
windows.title("以图搜图 v1.0")
windows.geometry('1300x600')
#window = Frame(windows,width=1300,height=600)
#window.pack(expand=True,fill=BOTH)
# Canvas of wafer
canvas_wafer = Canvas(master=windows)

#canvas_wafer.pack(expand=True,fill=BOTH)







# 第一行
label_1 = Label(canvas_wafer, text='输入硅圆SCRIBE_ID编号，搜索相似缺陷纹理硅圆图片：', cursor='circle')
#label_1.pack(side='left')
label_1.grid(column=0, row=0, sticky='w')
entry_scribe = Entry(canvas_wafer, width=25)
#entry_scribe.pack(side='right')
entry_scribe.grid(column=1,row=0)

# 第二行
label_2 = Label(canvas_wafer, text='搜索范围时限长度设置：', cursor='circle')
#label_2.pack(side='left')
label_2.grid(column=0,row=1,sticky='w')
combo_month = Combobox(canvas_wafer, width=23)
combo_month['values'] = ('一个月内','三个月内','六个月内','九个月内','十二个月内')
combo_month.current(1)
#combo_month.pack(side='right')
combo_month.grid(column=1, row=1)

# 第三行
label_ratio = Label(canvas_wafer, text='相关系数阈值（ 0 ~ 1.0 ）：', cursor='circle')
#label_1.pack(side='left')
label_ratio.grid(column=0, row=2, sticky='w')
entry_ratio = Entry(canvas_wafer, width=25)
#entry_scribe.pack(side='right')
entry_ratio.grid(column=1,row=2)

# 查找相似硅圆函数
dict_hash = np.load('//Jkfs/营运系统/器件与电性测试工程部/系统内跨部门工作区/项目类/EDA/以图搜图/dict_hash.npy',allow_pickle=True).item()
def find_result(dict_hash, str_scribe, float_limit):
    def calc_corr(a, b):
        # Correlation efficient calculas
        if len(a)*len(b) == 0:
            return 0
        else:
            a_avg = sum(a)/len(a)
            b_avg = sum(b)/len(b)
            cov_ab =  sum([(x-a_avg)*(y-b_avg) for x,y in zip(a,b)])
            sq = sum([(x-a_avg)**2 for x in a])*sum([(x-b_avg)**2 for x in b])
        if sq == 0:
            return 0
        else:
            return cov_ab/(sq**0.5)

    # Find correlated result to dictionary
    dict_result = {}
    list_input = dict_hash[str_scribe]
    if len(list_input)==0:
        float_ratio = 0
    else:
        float_ratio = sum(list_input)/len(list_input)
    def func_sub(list_row):
        if len(list_row)<=len(list_input):
            list_temp = list_input[:len(list_row)]
            float_corr = calc_corr(list_temp,list_row)
        else:
            list_temp = list_row[:len(list_input)]
            float_corr = calc_corr(list_temp,list_input)
        return float_corr
    for i in dict_hash:
        list_row = dict_hash[i]
        if len(list_row) > 0:
            float_diff = abs((sum(list_row)/len(list_row)) - float_ratio)
            float_corr = func_sub(list_row)
            if (float_corr > float_limit)  & (float_diff<0.1):
                dict_result[i] = [float_corr, float_diff]
    return dict_result

# Color config file
ColorTable = pd.read_excel('//Jkfs/营运系统/器件与电性测试工程部/系统内跨部门工作区/项目类/EDA/以图搜图/ColorTable.XLSX')
ColoRlist = np.array(ColorTable)
ColoRlist=ColoRlist.tolist()
new_blues=ColoRlist

# 搜索按钮
str_click = Label(canvas_wafer,  cursor='circle')
def func_click():
    str_click.grid_forget()
    #str_click.pack(side='left')
    str_click.configure(text='正在搜索'+combo_month.get()+entry_scribe.get()+'相关系数大于'+entry_ratio.get()+ '的相似图片......',wraplength = 1080)
    str_click.grid(column=0,row=6,sticky='w',columnspan=3)
    dict_result = find_result(dict_hash, entry_scribe.get(), float(entry_ratio.get()))
    int_num = int_column = 0
    int_row = 10
    for i in dict_result: 
        Sales = pd.read_csv('//Jkfs/营运系统/器件与电性测试工程部/系统内跨部门工作区/项目类/EDA/以图搜图/0905/' + i)
        #%%
        Summary = Sales.pivot_table(index = 'DIE_X', columns = 'DIE_Y', values = 'SOFTBIN', aggfunc = np.sum)
        #%%

        ListMap = np.array(Summary)
        ListMap = ListMap.tolist()
        #%%
        plt.figure(dpi=30,figsize=(10,10))
        #%%
        sns.set(style='white',rc = {'figure.figsize':(20,20)})
        #%%
        ax = sns.heatmap(ListMap,
                        cmap = new_blues, # 指定填充色
                        linewidths = 0.1, # 设置每个单元格边框的宽度
                        xticklabels=False,
                        yticklabels=False,
                        ax=None,
                        cbar=False,
                        cbar_ax=None)
        #%%


        #fig_name = 'A0C0054_01'+'.png'

        #fig_path = 'D:/CP_CP' + '/' + fig_name
        heatmap = ax.get_figure()
        #heatmap.savefig(fig_path, bbox_inches='tight',dpi = 400)
        #plt.clf()
        #plt.close()

        drawPic_f = heatmap
        drawPic_canvas = FigureCanvasTkAgg(drawPic_f, master=canvas_pic)
        drawPic_canvas.draw()
        drawPic_canvas.get_tk_widget().grid(row=int_row,column=int_column)
        #drawPic_canvas.pack(side = 'left')
        int_num = int_num + 1
        int_column = int_num % 6
        int_row = int_num // 6 +10
        #drawPic_a = drawPic_f.add_subplot(111)
        #drawPic_a = heatmap
        #drawPic_canvas.draw()
        
        
        fig_name = i + '.png'
        fig_path = str_file + '/' + fig_name
        heatmap.savefig(fig_path, bbox_inches='tight',dpi = 400)
     

        windows.update()
    str_click.configure(text='搜索完成，结果SCRIBE_ID列表：' + str(list(dict_result.keys())) ,wraplength = 1080)

    str_done = Label(canvas_wafer, text='图片已保存至: ' + str_file, cursor='circle')
    #str_click.pack(side='left')
    str_done.grid(column=0,row=7,sticky='w',columnspan=3)
        
    windows.update()


button_search = Button(canvas_wafer, text='搜索', command=func_click)
#button_search.pack(side='right')
button_search.grid(column=2,row=0)

#保存按钮
def func_save():
    str_click_save = Label(canvas_wafer, text='保存路径：', cursor='circle')
    #str_click_save.pack(side='left')
    str_click_save.grid(column=0,row=5, sticky='w')
    entry_value = StringVar()
    entry_save = Entry(canvas_wafer, textvariable=entry_value, width=25)
    #str_click_save.pack(side='right')
    entry_save.grid(column=1,row=5)
    def func_file():
        global str_file
        str_file = filedialog.askdirectory()
        entry_value.set(str_file)
    button_search = Button(canvas_wafer, text='选择路径', command=func_file)
    #button_search.pack(side='right')
    button_search.grid(column=2,row=5)





chk_state = BooleanVar()
chk_state.set(False) # Set check state
chk = Checkbutton(canvas_wafer, text="是否保存搜索结果图片", cursor='circle', var=chk_state, command=func_save)
#chk.pack(side='left')
chk.grid(column=0, row=4, sticky='w')

# 图片画布
frame_pic = Frame(canvas_wafer)
frame_pic.grid(column=0, row=9, sticky='w',columnspan=9)
canvas_pic = Canvas(master=frame_pic,width=3000,height=4000,  confine=True)
canvas_pic.configure(scrollregion=canvas_pic.bbox("all"))
def resize(event):
    canvas_pic.config(width=event.width-100,height=event.height-100)
canvas_pic.bind('<Configure>',resize)
#canvas_pic.grid(column=0, row=6, sticky='w',columnspan=6)

# 滚轮
# 滚动条
scrollBar = Scrollbar(frame_pic)
scrollBarx = Scrollbar(frame_pic, orient=HORIZONTAL)

#滚动条初始化（scrollBar为垂直滚动条，scrollBarx为水平滚动条）

# canvas_pic.config(scrollregion=(0,0,1300,1400))
#,width=1300,height=1400, ,
#靠右，充满Y轴
scrollBar.pack(side='right',fill=Y)
#靠下，充满X轴
scrollBarx.pack(side='bottom',fill=X)


scrollBar.config(command=canvas_pic.yview)
scrollBarx.config(command=canvas_pic.xview)
canvas_pic.config(xscrollcommand=scrollBarx.set, yscrollcommand=scrollBar.set)
def process_wheel(event):
    a = int(-(event.delta)/60)
    canvas_pic.yview_scroll(a,'units')

canvas_pic.bind('<MouseWheel>',process_wheel)
windows.bind('<MouseWheel>',process_wheel)

canvas_pic.pack(expand='yes',fill='both')
canvas_wafer.pack(expand=True,fill=BOTH)

scrollBar.config(command=canvas_pic.yview)
scrollBarx.config(command=canvas_pic.xview)






windows.update()
windows.mainloop()


In [ ]:
Traceback (most recent call last):
  File "CP2CP.py", line 22, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "seaborn\__init__.py", line 2, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "seaborn\rcmod.py", line 4, in <module>
  File "_distutils_hack\__init__.py", line 88, in create_module
  File "importlib\__init__.py", line 126, in import_module
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "setuptools\__init__.py", line 10, in <module>
  File "_distutils_hack\__init__.py", line 88, in create_module
  File "importlib\__init__.py", line 126, in import_module
ModuleNotFoundError: No module named 'setuptools._distutils'
[14120] Failed to execute script 'CP2CP' due to unhandled exception!





In [ ]:
Traceback (most recent call last):
  File "CP2CP.py", line 22, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "seaborn\__init__.py", line 2, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "seaborn\rcmod.py", line 4, in <module>
  File "_distutils_hack\__init__.py", line 88, in create_module
  File "importlib\__init__.py", line 126, in import_module
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "setuptools\__init__.py", line 10, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "distutils\core.py", line 16, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "distutils\dist.py", line 19, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "distutils\util.py", line 15, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "distutils\spawn.py", line 15, in <module>
ImportError: cannot import name 'log' from 'setuptools._distutils' (C:\Users\LUOYIN~1\AppData\Local\Temp\_MEI91042\setuptools\_distutils\__init__.pyc)
[2824] Failed to execute script 'CP2CP' due to unhandled exception!






In [ ]:
Traceback (most recent call last):
  File "CP2CP.py", line 15, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "seaborn\__init__.py", line 2, in <module>
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "seaborn\rcmod.py", line 4, in <module>
  File "_distutils_hack\__init__.py", line 88, in create_module
  File "importlib\__init__.py", line 126, in import_module
  File "PyInstaller\loader\pyimod02_importers.py", line 499, in exec_module
  File "setuptools\__init__.py", line 10, in <module>
  File "_distutils_hack\__init__.py", line 88, in create_module
  File "importlib\__init__.py", line 126, in import_module
ModuleNotFoundError: No module named 'setuptools._distutils'
[12412] Failed to execute script 'CP2CP' due to unhandled exception!

